<a href="https://colab.research.google.com/github/ajaypisharody/Coupa_BDS_test/blob/main/BDS_Assessment_Coupa_Software.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Task description

Historically black colleges and universities (HBCUs) are institutions of higher education in
the United States established with the intention of primarily serving the African-American
community.


Use this exact URL to see the list of certified HBCUs:
https://nces.ed.gov/COLLEGENAVIGATOR/?s=all&sp=4&pg=1


The expected record count is approximately 100-150 (when all pages are scraped). If your
scraper returns more records than this, you might have the incorrect site parameters. Please
refer to the above URL exactly as it is.
Using Python as the programming language, your task is to scrape the site for all HBCU
certified colleges/universities and generate a csv file containing the following data points and
data for all scraped colleges/universities. You are free to use any Python libraries of your choice.
1. Name
2. Street
3. City
4. State
5. Zip
6. Phone
7. Website
8. Type
9. Awards
10. Campus Setting
11. Campus Housing
12. Student Population
13. Student to Faculty ratio


In [ ]:
!pip install requests
!pip install html5lib
!pip install bs4

In [114]:
import requests
from bs4 import BeautifulSoup
import csv
import pandas as pd

URL = "https://nces.ed.gov/COLLEGENAVIGATOR/?s=all&sp=4&pg=1"
r = requests.get(URL)

soup = BeautifulSoup(r.content, 'html5lib')

table = soup.find('table', attrs = {'id':'ctl00_cphCollegeNavBody_ucResultsMain_tblResults'})
info=soup.find('td', attrs={'class':'infobutton'})
names=soup.find('h2',attrs={'id':'ctl00_cphCollegeNavBody_ucResultsMain_tblResults'})
row_nam=[]
urls = []
for link in table.find_all('a'):
 urls.append("https://nces.ed.gov/COLLEGENAVIGATOR"+link.get('href'))
for j in table.find_all('td'):
 row_data = j.find_all('h2')
 row_nam.append([i.text for i in row_data])

names=pd.DataFrame(row_nam)
names=names.dropna()
names=names.drop_duplicates()

row_det=[]
for j in table.find_all('tr'):
 row_data = j.find_all('td', attrs={'class':'sra'})
 row_det.append([i.text for i in row_data])

details=pd.DataFrame(row_det)
details=details.dropna()
details=details.drop_duplicates()

weblinks=pd.DataFrame(urls)
weblinks=weblinks.iloc[::2]


import re
row_add=[]
streets=[]
states=[]
zips=[]
extras=[]
for j in table.find_all('tr'):
  row_data= j.find_all('td', attrs={'class':'pbe'})
  for r in row_data:
    x = re.sub("^<td.*</h2>", "", str(r))
    x=re.sub("</td>$","",x)
    st, ct, s= x.split(',')
    streets.append(st+ct)
    extras.append(s)
    
    sss,z=s.rsplit(None,1)
    zips.append(z)

street=pd.DataFrame(streets)
street=street.drop_duplicates() 



extra=pd.DataFrame(extras) 
extra=extra.drop_duplicates() 

zip=pd.DataFrame(zips) 
zip=zip.drop_duplicates() 

state_split=[]

s=extra.values.tolist()
for i in s:
  ss,z=str(i).rsplit(None,1)
  state_split.append(ss)

state=pd.DataFrame(state_split)




details.insert(0,"Names",names.values)
details.insert(1,"Street",street.values)
details.insert(2,"State",state.values)
details.insert(3,"zip",zip.values)
details.insert(5,"URL",weblinks.values)



details.columns=['Name','Street','State','Zip','Phone','Website','Type','Awards','Campus Setting','Campus Housing','Student Population','Student to Faculty ratio']

details.to_csv("HBCU.csv")
print("HBCU.csv created with college details")

HBCU.csv created with college details
